In [1]:
import numpy as np
import matplotlib.pyplot as plt  
%matplotlib inline
# from IPython.display import display, HTML

from pykat import finesse
from pykat.commands import *
pykat.init_pykat_plotting(dpi=90)

from pprint import pprint as ppprint

                                              ..-
    PyKat 1.2.2           _                  '(
                          \`.|\.__...-""""-_." )
       ..+-----.._        /  ' `            .-'
   . '            `:      7/* _/._\    \   (
  (        '::;;+;;:      `-"' =" /,`"" `) /
  L.        \`:::a:f            c_/     n_'
  ..`--...___`.  .    ,
   `^-....____:   +.      www.gwoptics.org/pykat



In [2]:
# base model that holds the components, run detection models on top of this
basekat = finesse.kat()
#basekat.verbose = False

basecode = """
# no spaces in model, except inside cavity

l laser 1 0 0 nbs1 # power = 1W
bs BS 0.5 0.5 0 45 nbs1 nbs2 nbs3 nbs4 # phi = 0

s sdbs1 0 nbs4 ndbs1 # zero length space
dbs isol ndbs1 ndbs2 ndbs3 ndbs4  # dbs doesn't req R T phi alpha
s sdbs2 0 ndbs2 nc0 # zero length space

m m1 0.5 0.5 0 nc0 nc1 # R = 0.5 = T, phi = 0
s sc1 1 nc1 nc2 # L = 1m
nle nle1 -10 0 nc2 nc3 # r = -10, sqz angle = 0
s sc2 1 nc3 nc4 # L = 1m
m m2 1 0 0 nc4 nc5 # R = 1, T = 0, phi = 0

noxaxis
"""

basekat.parse(basecode)

Parsing `nle nle1 -10 0 nc2 nc3` into pykat object not implemented yet, added as extra line.


In [3]:
ppprint((basekat.components, basekat.detectors, basekat.commands))

({'BS': <pykat.components.beamSplitter_2 (BS) at 0x7f56c0605908>,
  'isol': <pykat.components.dbs_4 (isol) at 0x7f568c2a1160>,
  'laser': <pykat.components.laser_1 (laser) at 0x7f56c0605898>,
  'm1': <pykat.components.mirror_6 (m1) at 0x7f568c2a1940>,
  'm2': <pykat.components.mirror_9 (m2) at 0x7f568c2a7828>,
  'sc1': <pykat.components.space_7 (sc1) at 0x7f568c2a70f0>,
  'sc2': <pykat.components.space_8 (sc2) at 0x7f568c2a7400>,
  'sdbs1': <pykat.components.space_3 (sdbs1) at 0x7f568c291278>,
  'sdbs2': <pykat.components.space_5 (sdbs2) at 0x7f568c291fd0>},
 {},
 {})


In [4]:
kat1 = deepcopy(basekat)

kat1code = """

qhd qn 180 nbs2 nbs3
fsig noise 1 # noise at 1 Hz

# by default noise enters through every open port, lossy optic, or squeezer
# -- Quantum noise inputs --
#   |- %10-.10s - laser Node LASER NOISE
#   |- %10-.10s - m2 Node OPEN PORT
#   |- %10-.10s - BS Node OPEN PORT
#   |- %10-.10s - BS Node OPEN PORT
#   |- %10-.10s - isol Node OPEN PORT
#   |- %10-.10s - isol Node OPEN PORT
# can instead specify exactly where through "vacuum" command
# e.g. vacuum BS isol
printnoises

# xaxis nle r lin -10 0 10 1000 # to spot threshold
# yaxis abs:deg
"""

kat1.parse(kat1code)

Parsing `printnoises` into pykat object not implemented yet, added as extra line.


In [5]:
ppprint((kat1.components, kat1.detectors, kat1.commands))

({'BS': <pykat.components.beamSplitter_11 (BS) at 0x7f568c2b4358>,
  'isol': <pykat.components.dbs_13 (isol) at 0x7f568c2b23c8>,
  'laser': <pykat.components.laser_10 (laser) at 0x7f56c06059b0>,
  'm1': <pykat.components.mirror_15 (m1) at 0x7f568c2b2898>,
  'm2': <pykat.components.mirror_18 (m2) at 0x7f568c2bba90>,
  'sc1': <pykat.components.space_16 (sc1) at 0x7f568c2bb4e0>,
  'sc2': <pykat.components.space_17 (sc2) at 0x7f568c2bb7b8>,
  'sdbs1': <pykat.components.space_12 (sdbs1) at 0x7f568c2b20f0>,
  'sdbs2': <pykat.components.space_14 (sdbs2) at 0x7f568c2b25c0>},
 {'qn': <pykat.detectors.pykat.detectors.qhd_1 object at 0x7f568c2b2390>},
 {})


In [6]:
out1 = kat1.run()
print(out1.stdout)

--------------------------------------------------------------
Running kat - Started at 2020-08-20 11:46:35.184872
Used Finesse 2.3.1 at /home/james/Code/cbrOzGrav/Finesse/kat

Finished in 0.01085 seconds

------------------------------------------------------------------------
                     FINESSE 2.3.1            (build 2.3.1.11-ga3cc55b2)
       o_.-=.        Frequency domain INterferomEter Simulation SoftwarE
      (\'".\|        06.07.2020         http://www.gwoptics.org/finesse/
      .>' (_--.      
   _=/d   ,^\        Input file /tmp/tmpztn36l2h.kat,
  ~~ \)-'   '        Output file /tmp/tmpztn36l2h.out,
     / |             Gnuplot file /tmp/tmpztn36l2h.gnu 
    '  '                                        Thu Aug 20 11:46:35 2020
------------------------------------------------------------------------
 'noxaxis' has been set, ignoring all other xaxis commands
 
-- Quantum noise inputs --
  |- %10-.10s - laser Node LASER NOISE
  |- %10-.10s - m2 Node OPEN PORT
  |- %10

100% | ETA:  0:00:00 | Calculating                                             

In [7]:
# Vaishali's test code to check nle installed
testkat = finesse.kat()
testcode = """
l l1 1 0 0 n1
s s0 1 ni na
nle nle1 -10 0 na nb
s s1 1 nb n4
bs bs1 0.5 0.5 0 0 n1 n2 n3 n4
qhd qn 180 n2 n3
fsig noise 1
noxaxis
printnoises
"""
testkat.parse(testcode)

Parsing `nle nle1 -10 0 na nb` into pykat object not implemented yet, added as extra line.
Parsing `printnoises` into pykat object not implemented yet, added as extra line.


In [8]:
out = testkat.run()
print(out.stdout)

--------------------------------------------------------------
Running kat - Started at 2020-08-20 11:46:35.216751
Used Finesse 2.3.1 at /home/james/Code/cbrOzGrav/Finesse/kat

Finished in 0.0120752 seconds

------------------------------------------------------------------------
                     FINESSE 2.3.1            (build 2.3.1.11-ga3cc55b2)
       o_.-=.        Frequency domain INterferomEter Simulation SoftwarE
      (\'".\|        06.07.2020         http://www.gwoptics.org/finesse/
      .>' (_--.      
   _=/d   ,^\        Input file /tmp/tmpheakjm0q.kat,
  ~~ \)-'   '        Output file /tmp/tmpheakjm0q.out,
     / |             Gnuplot file /tmp/tmpheakjm0q.gnu 
    '  '                                        Thu Aug 20 11:46:35 2020
------------------------------------------------------------------------
 'noxaxis' has been set, ignoring all other xaxis commands
 
-- Quantum noise inputs --
  |- %10-.10s - l1 Node LASER NOISE
  |- %10-.10s - bs1 Node OPEN PORT
  |- %10

100% | ETA:  0:00:00 | Calculating                                             